# Logistic Regression Handson

Import Library

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import math
import scipy

In [ ]:
#Reading data
data = pd.read_csv('bank', sep=';')
data

Problem Statement : If the person is going to buy the insurance or not based on various features provided in the data

In [ ]:
data.shape

In [ ]:
data.info() #There are null values

In [ ]:
#Finding outliers using boxplot
cols = list(data.columns)
for col in cols:
    if data[col].dtype=='float64':
        sns.boxplot(data[col])
        plt.xlabel(col)
        plt.ylabel('Count')
        plt.show()

In [ ]:
#Finding outlier using upper fence and lower fence
cols = ['age', 'duration', 'campaign', 'cons.conf.idx'] #List of columns to remove outliers of

#Finding outlier and removing it for age column
q1 = data.age.quantile(0.25)
q3 = data.age.quantile(0.75)
iqr = q3 - q1
upper_fence = q3 + 1.5*iqr
lower_fence = q1 - 1.5*iqr
data = data[(data.age>lower_fence) & (data.age<upper_fence)]


#plotting boxplot after removing outlier
sns.boxplot(data.age)
plt.show()



In [ ]:
#Finding outlier and removing it for duration column
q1 = data.duration.quantile(0.25)
q3 = data.duration.quantile(0.75)
iqr = q3 - q1
upper_fence = q3 + 1.5*iqr
lower_fence = q1 - 1.5*iqr
data = data[(data.duration>lower_fence) & (data.duration<upper_fence)]



#plotting boxplot after removing outlier
sns.boxplot(data.duration)
plt.show()

In [ ]:
#Finding outlier and removing it for campaign column
q1 = data.campaign.quantile(0.25)
q3 = data.campaign.quantile(0.75)
iqr = q3 - q1
upper_fence = q3 + 1.5*iqr
lower_fence = q1 - 1.5*iqr
data = data[(data.campaign>lower_fence) & (data.campaign<upper_fence)]

#plotting boxplot after removing outlier
sns.boxplot(data.campaign)
plt.show()

Feature Selection

In [ ]:
#Taking two categorical variables which we want to compare if they are related
#First categorize both columns into grouping, for age column use pd.cut(), it takes bins parameter to group by certain range
#Finding observed values using pd.crosstab function, it takes two columns as paramters
#Finding expected value using chi2_contingency function and it takes observed value as input
#Finding the chi square statistics, the formula is summation(o-e**2/e)
#Finding degree of freedom, no.of.rows-1 * no.of.columns-1 of selected columns and rows only
#Setting alpha value
#Finding the critical value, it requires b














#Creating bins/grouping by for age column

data['age'] = pd.cut(data['age'], bins = [0,50,100])
data['age'] = data['age'].astype('str')
data

In [ ]:
#EDA - 83% of customers are in age group 0-50 and 16% in age group 50-100
data.age.value_counts()/len(data.age)*100

In [ ]:
#Chi-Square test : It checks if 2 categorical variables are related or not
#We find the observed values and find the expected values and compare them and check if there is any depedency between them or not.

#Forming Hypothesis
#Null Hypothesis : Both variables are related
#Alternate Hypothesis : Both variables are not related

#importing libraries
from scipy.stats import chisquare
from scipy.stats import chi2
from scipy.stats import chi2_contingency



#Creating contingency table for age and y column
contingency_table = pd.crosstab(data['age'], data['y'])
contingency_table
#Inference from contingency table - 
''': People within age group 0-50 have people 26k not bought the insurance
: People within age group 0-50 have people 2k bought the insurance
: People within age group 50-100 have people 5k who not bought the insurance
: People within age group 50-100 have people 0.5k who bought the insurance'''
    


#dataframe object method values will convert dataframe into array format
observed_values = contingency_table.values
#observed values
observed_values


#You get expected_frequency
statistics = chi2_contingency(observed_values)
expected_values = statistics[3]
#expected values
expected_values

In [ ]:
chi_square = sum([(o-e)**2/e for o, e in zip(observed_values, expected_values)])

chi_square
chi_square[0] + chi_square[1]

In [ ]:
#Without for loop
chi_square = np.sum(((observed_values - expected_values) ** 2) / expected_values)
chi_square

In [ ]:
# Calculating chi-square contributions for each pair of observed and expected values
chi_square_contributions = [(o - e) ** 2 / e for o, e in zip(observed_values.ravel(), expected_values.ravel())] #ravel method of array object will flatten or convert the array into 1D.

# Sum up all the chi-square contributions to get the total chi-square statistic
chi_square = np.sum(chi_square_contributions)

print(chi_square)

In [ ]:
#finding degree of freedom
num_of_rows = len(contingency_table.iloc[0:2,0])
num_of_rows


In [ ]:
#finding degree of freedom
num_of_columns = len(contingency_table.iloc[:,0:])
num_of_columns



In [ ]:
#degree of freedom
dof = (num_of_rows-1)*(num_of_columns-1)
dof

In [ ]:
alpha = 0.05

In [ ]:
#Finding critical value
critical_value = scipy.stats.chi2.ppf(1-alpha, df=dof)
#Inverse cumulative distribution - critical value
critical_value
#We will compare the critical with chi square statistics

In [ ]:
#Summarizing hypothesis
#if chi square statistics >= critical value -----> reject null hypothesis
print(f'Chi Square Statistics : {chi_square}\nCritical Value {critical_value}')

In [ ]:
#Finding pvalue for each column
l = []
cols = list(data.columns)
cols.remove('y')

for col in cols:
    if data[col].dtype == 'object':
        cross_table = pd.crosstab(data[col], data['y'])
        observed_values = cross_table.values
        expected_values = chi2_contingency(observed_values)[3]
        chi_square = sum([(o-e)**2/e for o, e in zip(observed_values,expected_values)])
        chi_square_statistics = chi_square[0] + chi_square[1]
        no_of_rows = observed_values.shape[0]
        no_of_cols = observed_values.shape[1]
        dof = (no_of_rows-1)*(no_of_cols-1)
        alpha = 0.05
        critical_value = chi2.ppf(q= 1-alpha, df=dof)
        pvalue = 1 - chi2.cdf(x=chi_square_statistics, df=dof)
        l.append({'Feature' : col, 'Pvalue' : pvalue})

feature_pvalue = pd.DataFrame(l)
print(feature_pvalue)

#Since loan's pvalue is 0.2573652 i.e. > alpha or 0.05 we will drop that column 
data.drop(columns = ['loan'], inplace = True)


In [195]:
#label encoding
col_list = []
for col in data.columns:
    if[(data[col].dtype=='object')&(col!='y')]:
        col_list.append(col)
    from sklearn.preprocessing import LabelEncoder
    le = LabelEncoder()
    for i in col_list:
        data[i] = le.fit_transform(data[i])
        

Variance Inflation Factor

In [205]:
data.drop('poutcome', axis=1, inplace=True)


In [206]:
#Multiconnearity : It refers to high correlation between two independent variables or predictors. 
#Example; Date of Birth and Age, both of this columns are example of multiconnearity, because any one is enough to know the age of a person. If we keep both columsn while train and test, the model's performance/accuracy willl be bad, because there shouldn't exist two variables high correlation.
#We can find this multiconnearity using Variance Inflation Factor. We will check if certain independent column depdent on another independent column


from statsmodels.stats.outliers_influence import variance_inflation_factor

col_list = []
for col in data.columns:
    if (data[col].dtype!='object') & (col!='y'):
        col_list.append(col)

    
x = data[col_list]
vif_data = pd.DataFrame()
vif_data['Features'] = x.columns
vif_data['VIF'] = [variance_inflation_factor(x.values, i) for i in range(len(x.columns))]
#We have a dataframe with columns with their respective VIF value. 
#Remember we need to drop all columns which has greater than 5 VIF value, as we drop one highest VIF column, the values changes for all columns.
#If other columns' VIF value changes it means the other columns are negatively or positively correlated.
#We need to drop one column at a time as the changes will applied after dropping one column
vif_data



,Features,VIF
0,age,1.249024
1,job,2.056418
2,marital,4.223433
3,education,3.976407
4,default,1.302323
5,housing,2.074821
6,contact,2.043274
7,month,4.158586
8,day_of_week,2.836433
9,duration,2.896304


In [215]:
#Building Model. Feature Engineering/Selection done, splitting x and y i.e. independent and dependent variable
a = vif_data['Features']
x = data[a]
x

y = data['y']
y

In [216]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size = 0.2, random_state = 0)

In [217]:
#Standardizing and scaling the data, it helps the ML momdel to run efficiently and faster
#Our data can be in different units and forms, so we need to standarize it.
#scaling means making the data linear/uniform in units and making it easier for model to run faster and efficiently
#Say, you have 214,155,1005151,201e-02, 2011, 241 and 0.2, 0.4, 0.5, 0.6, 0.7, the decimal values has uniformity while others nmbers don't, so scaling will just do that, make it uniform and make sure same units are there. 

from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
x_train = ss.fit_transform(x_train)
x_test = ss.fit_transform(x_test)


In [218]:
#Building model
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(x_train,y_train)

LogisticRegression()

In [219]:
#Predicting for x_test data
y_pred = lr.predict(x_test)

In [222]:
#Checking TP, TN
from sklearn.metrics import *
confusion_matrix(y_pred,y_test)

array([[6287,  474],
       [  69,   97]], dtype=int64)

In [223]:
#Checking if data is balanced
data['y'].value_counts()


0    31799
1     2835
Name: y, dtype: int64

In [229]:
#Finding Recall, Precision, F-Beta because data is imbalanced, 1 is 2835 and 0 is 31799

Recall = 6287/(6287+69) #TP/TP+FN
print(Recall)

Precision = 6287/(6287+474) #TP/TP+FP
print(Precision)

F_Beta = 2*Precision*Recall/1 * Precision + Recall # (1+B^2) * Precision * Recall/B^2 * Precision + Recall, we have taken B = 1, hence B^2 = 1
F_Beta

0.9891441157960982
0.9298920278065375


2.6997683341404835

In [232]:
#If you want to do with accuracy the formula is as below: **Remember the data is imbalanced data we don't find performance using accuracy metric.
accuracy = accuracy_score(y_pred, y_test)
accuracy

0.9216110870506713